# Use Cases

## 1. Q&A with RAG

## 2. Extracting Structured Output

## 3. Chatbots

## 4. Tool use and Agents

### Tool 사용하기

In [14]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(
    lang="en",                          # 검색할 언어
    top_k_results=1,                    # 상위 k개 결과 반환
    doc_content_chars_max=100,          # 로드할 문자 수
    load_all_available_meta=False       # 메타데이터 로드 여부
)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

print(tool.name)
print(tool.description)
print(tool.args)

wikipedia
A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
{'query': {'title': 'Query', 'type': 'string'}}


In [15]:
tool.run({"query":"langchain"})

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications '

In [10]:
tool.run("langchain")

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications '

### Tool 만들기

In [1]:
from langchain_core.tools import tool

@tool
def multiply(int1:int, int2:int):
    """두 정수를 곱하는 함수"""
    return int1 * int2

print(multiply)

name='multiply' description='multiply(int1: int, int2: int) - 두 정수를 곱하는 함수' args_schema=<class 'pydantic.v1.main.multiplySchema'> func=<function multiply at 0x000001988B3B7C70>


In [4]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
multiply(int1: int, int2: int) - 두 정수를 곱하는 함수
{'int1': {'title': 'Int1', 'type': 'integer'}, 'int2': {'title': 'Int2', 'type': 'integer'}}


In [19]:
multiply.invoke({"int1":2, "int2":6})

12

### Tool 호출(프롬프트 이용)

In [26]:
from langchain.tools.render import render_text_description 

rendered_tools = render_text_description([multiply])
rendered_tools

'multiply: multiply(int1: int, int2: int) - 두 정수를 곱하는 함수'

In [42]:
template = """\
# INSTRUCTION
TOOL은 도구의 이름과 설명에 대한 내용입니다. 
USER 내용을 기반으로 도구의 이름과 입력값을 KEY에 따라 JSON 형식으로 출력하세요.

# TOOL: {rendered_tools}
# USER: {input}
# KEY: name, arguments
"""

In [43]:
# Tool을 LLM에 전달
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from dotenv import load_dotenv
load_dotenv()

prompt = PromptTemplate.from_template(template)
model = ChatOpenAI(model_name="gpt-3.5-turbo")
chain = prompt | model | JsonOutputParser()
chain.invoke({"input": "125 곱하기 333이 뭐예요", "rendered_tools":rendered_tools})

{'name': 'multiply', 'arguments': {'int1': 125, 'int2': 333}}

In [44]:
# 출력까지 연결
from operator import itemgetter

chain = prompt | model | JsonOutputParser() | itemgetter("arguments") | multiply
chain.invoke({"input": "125 곱하기 333이 뭐예요", "rendered_tools":rendered_tools})

41625

In [46]:
# 도구 만들기
from langchain_core.tools import tool

@tool
def multiply(int1:int, int2:int):
    """두 정수를 곱하는 함수"""
    return int1 * int2

@tool
def add(int1:int, int2:int):
    """두 정수를 더하는 함수"""
    return int1 + int2 

@tool
def exponentiate(base:int, exponent:int):
    """거듭제곱하는 함수"""
    return base**exponent

tools = [multiply, add, exponentiate]

In [47]:
from langchain.tools.render import render_text_description 

rendered_tools = render_text_description(tools)
rendered_tools

'multiply: multiply(int1: int, int2: int) - 두 정수를 곱하는 함수\nadd: add(int1: int, int2: int) - 두 정수를 더하는 함수\nexponentiate: exponentiate(base: int, exponent: int) - 거듭제곱하는 함수'

In [48]:
template = """\
# INSTRUCTION
TOOL은 도구의 이름과 설명에 대한 내용입니다. 
USER 내용을 기반으로 도구의 이름과 입력값을 KEY에 따라 JSON 형식으로 출력하세요.

# TOOL: {rendered_tools}
# USER: {input}
# KEY: name, arguments
"""

In [49]:
# Tool을 LLM에 전달
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from dotenv import load_dotenv
load_dotenv()

prompt = PromptTemplate.from_template(template)
model = ChatOpenAI(model_name="gpt-3.5-turbo")
chain = prompt | model | JsonOutputParser()
chain.invoke({"input": "125 곱하기 333이 뭐예요", "rendered_tools":rendered_tools})

{'name': 'multiply', 'arguments': {'int1': 125, 'int2': 333}}

In [50]:
chain.invoke({"input": "125 더하기 333이 뭐예요", "rendered_tools":rendered_tools})

{'name': 'add', 'arguments': {'int1': 125, 'int2': 333}}

In [51]:
chain.invoke({"input": "2의 네제곱이 뭐에요", "rendered_tools":rendered_tools})

{'name': 'exponentiate', 'arguments': {'base': 2, 'exponent': 4}}

In [59]:
def tool_chain(model_output):
    tool_map = {tool.name: tool for tool in tools}
    chosen_tool = tool_map[model_output["name"]]

    return itemgetter("arguments") | chosen_tool

In [60]:
chain = prompt | model | JsonOutputParser() | tool_chain
chain.invoke({"input": "125 곱하기 333이 뭐예요", "rendered_tools":rendered_tools})    

41625

In [61]:
chain.invoke({"input": "125 더하기 333이 뭐예요", "rendered_tools":rendered_tools})

458

In [62]:
chain.invoke({"input": "2의 네제곱이 뭐에요", "rendered_tools":rendered_tools})

16

In [75]:
chain.invoke({"input": "125 더하기 333을 한 후, 그 결과와 2를 곱해주세요", "rendered_tools":rendered_tools})

916

### Tool 호출(함수 이용)

In [63]:
# Tool을 LLM에 전달
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

model = ChatOpenAI(model_name="gpt-3.5-turbo")
model_with_tool = model.bind_tools([multiply])

In [65]:
# 잘 전달되는지 확인
query = model_with_tool.invoke("125 곱하기 333이 뭐예요")
query

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_sstOD4EW1JKtTPkppdeMVcGD', 'function': {'arguments': '{"int1":125,"int2":333}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 77, 'total_tokens': 96}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e6683206-0ec7-4eb4-b3cd-bee72391d06d-0', tool_calls=[{'name': 'multiply', 'args': {'int1': 125, 'int2': 333}, 'id': 'call_sstOD4EW1JKtTPkppdeMVcGD'}])

In [66]:
query.tool_calls

[{'name': 'multiply',
  'args': {'int1': 125, 'int2': 333},
  'id': 'call_sstOD4EW1JKtTPkppdeMVcGD'}]

In [45]:
# 전체 확인
chain = model_with_tool | (lambda x: x.tool_calls[0]["args"]) | multiply
chain.invoke("125 곱하기 333이 뭐예요")

41625

In [81]:
# 도구 만들기
from langchain_core.tools import tool

@tool
def multiply(int1:int, int2:int):
    """두 정수를 곱하는 함수"""
    return int1 * int2

@tool
def add(int1:int, int2:int):
    """두 정수를 더하는 함수"""
    return int1 + int2 

@tool
def exponentiate(base:int, exponent:int):
    """거듭제곱하는 함수"""
    return base**exponent

tools = [multiply, add, exponentiate]

In [88]:
# Tool을 LLM에 전달
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

model = ChatOpenAI(model_name="gpt-3.5-turbo")
model_with_tool = model.bind_tools(tools)

In [83]:
# 잘 전달되는지 확인
query = model_with_tool.invoke("125 더하기 333을 한 후, 그 결과와 2를 곱해주세요")
query

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ZdVV0x5ng5PAcTeLefosPxvK', 'function': {'arguments': '{"int1": 125, "int2": 333}', 'name': 'add'}, 'type': 'function'}, {'id': 'call_0hi43anAKJD3JlThDuzvSpnv', 'function': {'arguments': '{"int1": 458, "int2": 2}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 169, 'total_tokens': 222}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5e2010b0-7e34-41a4-af07-0d03800aa8a8-0', tool_calls=[{'name': 'add', 'args': {'int1': 125, 'int2': 333}, 'id': 'call_ZdVV0x5ng5PAcTeLefosPxvK'}, {'name': 'multiply', 'args': {'int1': 458, 'int2': 2}, 'id': 'call_0hi43anAKJD3JlThDuzvSpnv'}])

In [84]:
query.tool_calls

[{'name': 'add',
  'args': {'int1': 125, 'int2': 333},
  'id': 'call_ZdVV0x5ng5PAcTeLefosPxvK'},
 {'name': 'multiply',
  'args': {'int1': 458, 'int2': 2},
  'id': 'call_0hi43anAKJD3JlThDuzvSpnv'}]

In [85]:
# 함수로 만들어 chain 만들기
from langchain_core.messages import AIMessage
def call_tools(message:AIMessage):
    """도구를 실행하는 함수"""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = message.tool_calls.copy()
    
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])

    return tool_calls

In [89]:
chain = model_with_tool | call_tools
chain.invoke("125 더하기 333을 한 후, 그 결과와 2를 곱해주세요")

[{'name': 'add',
  'args': {'int1': 125, 'int2': 333},
  'id': 'call_kyz5RrOWHpA4DFpeZq5BH6hW',
  'output': 458},
 {'name': 'multiply',
  'args': {'int1': 458, 'int2': 2},
  'id': 'call_S1mxwXriL1O0vcQZ1jyI5lBa',
  'output': 916}]

: 

### Agent

In [5]:
# 프롬프트 생성
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [6]:
# 도구 만들기
from langchain_core.tools import tool

@tool
def multiply(int1:int, int2:int):
    """두 정수를 곱하는 함수"""
    return int1 * int2

@tool
def add(int1:int, int2:int):
    """두 정수를 더하는 함수"""
    return int1 + int2 

@tool
def exponentiate(base:int, exponent:int):
    """거듭제곱하는 함수"""
    return base**exponent

tools = [multiply, add, exponentiate]

In [7]:
# LLM 생성
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

model = ChatOpenAI(model_name="gpt-3.5-turbo")

In [8]:
# Agent 생성
from langchain.agents import AgentExecutor, create_tool_calling_agent
agent = create_tool_calling_agent(model, tools, prompt)

In [9]:
# Agent 실행기 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

In [24]:
agent_executor.invoke(
    {"input": "3을 다섯 제곱하고, 그것을 12와 3의 합으로 곱한 다음, 전체 결과를 제곱하세요"}
)



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


243
Invoking: `add` with `{'int1': 12, 'int2': 3}`


15
Invoking: `multiply` with `{'int1': 243, 'int2': 15}`


3645
Invoking: `exponentiate` with `{'base': 3645, 'exponent': 2}`


13286025결과는 13,286,025입니다.

> Finished chain.


{'input': '3을 다섯 제곱하고, 그것을 12와 3의 합으로 곱한 다음, 전체 결과를 제곱하세요',
 'output': '결과는 13,286,025입니다.'}

In [25]:
agent_executor.invoke(
    {"input": "3을 다섯 제곱해줘"}
)



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


2433을 다섯 제곱하면 243이 됩니다.

> Finished chain.


{'input': '3을 다섯 제곱해줘', 'output': '3을 다섯 제곱하면 243이 됩니다.'}

### 인간 승인 기능 넣기

In [ ]:
from operator import itemgetter 

from langchain_core.messages import AIMessage 
from langchain_core.runnables import Runnable, RunnablePassthrough
